# Models

This notebook will demonstrate the basic features of AutoMPC for system ID modeling and model evaluation.

## Set-Up

As before, we begin by importing autompc.

In [1]:
import autompc as ampc

Loading AutoMPC...
Finished loading AutoMPC


To perform system identification, we need a dataset of trajectories to work with.  We will use the cartpole system, available from the `benchmarks` package, to generate our dataset.

In [2]:
from autompc.benchmarks import CartpoleSwingupBenchmark

benchmark = CartpoleSwingupBenchmark()

system = benchmark.system
trajs = benchmark.gen_trajs(seed=100, n_trajs=500, traj_len=200)

## Models

AutoMPC provides a variety of sytem ID models which can be used to learn the sytem dynamics.  Here, we will use an MLP model, but for a complete list see [here](https://autompc.readthedocs.io/en/latest/source/sysid.html#supported-system-id-models).

(**Note:** This will take several minutes to run depending on your hardware).

In [ ]:
from autompc.sysid import MLP

model = MLP(system)
model.train(trajs)

 12%|█▏        | 6/50 [00:19<02:26,  3.32s/it]

Now that we have trained our model, we can use it to make predictions.  Let's try predicting the next state from one of our training trajectories.  We first compute the model state at a certain point in the trajectory

In [ ]:
traj = trajs[0]
model_state = model.traj_to_state(traj[:100])

The model state contains the information the model needs to predict the next time step.  `model_state[:system.obs_dim]` is always equal to the most recent observation.  For the MLP, that's actually all there is to the model state, but some models require a larger state.  We can see the dimension of the model state by running

In [ ]:
model.state_dim

We can also check other properties of the model, such as whether it is differentiable and whether it is linear.

In [ ]:
print("Model is Differentiable? ", model.is_diff)
print("Model is Linear? ", model.is_linear)

For comparison, consider the ARX model.  We observe that, unlike the MLP model, the ARX model state size is larger than `system.obs_dim` since the model state includes the history of several observations.  Make sure to use the `traj_to_state` method to properly derive the model state.  We can also observe that the ARX model is linear, which means that it is suitable for use with LQR control.

In [ ]:
from autompc.sysid import ARX

model_arx = ARX(system)
model_arx.train(trajs)

In [ ]:
model_state_arx = model_arx.traj_to_state(traj[:100])
model_arx.state_dim

In [ ]:
model_arx.is_linear

We can use our current model state, and the control to make a prediction of the new model state

In [ ]:
pred_state = model.pred(model_state, traj[99].ctrl)
pred_state

Compare this to the true observation

In [ ]:
traj[100].obs

We can use the true observation to update our model state

In [ ]:
new_model_state = model.update_state(model_state, traj[99].ctrl, traj[100].obs)
new_model_state

For differentiable models, we can also get the Jacobian of the
model prediction

In [ ]:
pred_state, state_jac, ctrl_jac = model.pred_diff(model_state, traj[99].ctrl)
state_jac

## Graphing Model Accuracy

Let's train another, much smaller MLP model

In [ ]:
from autompc.sysid import MLP

model2 = MLP(system, n_hidden_layers=1, hidden_size_1=32, n_train_iters=50,
               nonlintype="relu")

model2.train(trajs)

Now, we'd like to compare this to our original model.  One convenient way to do this is by graphing the model prediction horizon over various prediction horizons.  AutoMPC provides tools for easily constructing this graph.  (**Note:** This may take a few minutes to run)

In [ ]:
import matplotlib.pyplot as plt
from autompc.graphs.kstep_graph import KstepPredAccGraph

graph = KstepPredAccGraph(system, trajs, kmax=20, metric="rmse")
graph.add_model(model, "Large MLP")
graph.add_model(model2, "Small MLP")

fig = plt.figure()
ax = fig.gca()
graph(fig, ax)
ax.set_title("Comparison of MLP models")
plt.show()

We can see that the small MLP has a larger prediction error at all time steps and its prediction error increases more rapidly over longer prediction horizons.